# <font color='blue'>Data Science Academy - Python Fundamentos - Capítulo 7</font>

## Download: http://github.com/dsacademybr

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


## Missão: Implementar um algoritmo para mover um robô do canto superior esquerdo para o canto inferior direito de uma grade.

## Nível de Dificuldade: Médio

## Premissas

* Existem restrições de como o robô se move?
     * O robô só pode se mover para a direita e para baixo
* Algumas células são inválidas (fora dos limites)?
     * Sim
* Podemos supor que as células inicial e final são células válidas?
     * Sim
* Isso é uma grade retangular? ou seja, a grade não é irregular?
     * Sim
* Haverá sempre uma maneira válida para o robô chegar ao canto inferior direito?
     * Não, retorno None
* Podemos assumir que as entradas são válidas?
     * Não
* Podemos supor que isso se encaixa na memória?
     * Sim

## Teste Cases

<pre>
o = célula válida
x = célula inválida

   0  1  2  3
0  o  o  o  o
1  o  x  o  o
2  o  o  x  o
3  x  o  o  o
4  o  o  x  o
5  o  o  o  x
6  o  x  o  x
7  o  x  o  o
</pre>

* Caso geral

```
Saída esperada = [(0, 0), (1, 0), (2, 0),
                  (2, 1), (3, 1), (4, 1),
                  (5, 1), (5, 2), (6, 2), 
                  (7, 2), (7, 3)]
```

* Nenhum caminho válido, por exemplo, linha 7, col 2 é inválido
* Nenhuma entrada
* Matriz vazia

## Solução

In [273]:
import random

class Grid(object):

    def find_path(self, matrix):
        
        path = None
        
        if (self.isValid(matrix)):
            
            self.map = matrix
            
            self.ymax = len(matrix) - 1
            self.xmax = len(matrix[0]) - 1
            
            self.origin = (0, 0)
            self.target = (self.ymax, self.xmax)
            
            path = self.pathFinding(0, [(self.target, 0)])

            
        return path
    
    
    # Find a unique path from origin to target on nodes
    # Assumption: there is at least one path
    def clearPath(self, index, temp_path, nodes):

        current_level = [node for node in nodes if node[1] == index]
        remain_nodes = [node for node in nodes if node[1] != index]
        
        if (temp_path == None):
            
            return None
        
        elif (len(temp_path) == 0):
            
            temp_path.append(current_level[0][0])
            
            temp_path = self.clearPath(index + 1, temp_path, remain_nodes)
            
            return temp_path
        
        else:
            
            for n in current_level:
                
                if (self.isAdjacent(temp_path[-1], n[0])):
                    
                    if (n[0] == (0, 0)):
                        
                        temp_path.append(n[0])
                        
                        return temp_path
                    
                    else:
                        
                        new_temp = temp_path.copy()
                        
                        new_temp.append(n[0])

                        new_temp = self.clearPath(index + 1, new_temp, remain_nodes)
                        
                        if (new_temp != None):
                        
                            return new_temp
                        
                        
                elif (n[0] == (0, 0)):
                    return None
 
    
    def isAdjacent(self, n0, n1):
        return (abs(n0[0] - n1[0] <= 1)) and (abs(n0[1]-n1[1] <= 1)) 
            
    # Mappping of adjacences from origin to target
    # clearPath find a unique path from mapping
    def pathFinding(self, index, path):
        
            current = path[index]

            if (current[0] == self.origin):
                
                path = self.clearPath(0, [], path)
                
                return path[::-1]

            else:

                candidates = self.getCandidates(current)
                
                valid_candidates = []

                for c in candidates:
                    if (self.map[ c[0][0] ][ c[0][1] ] == 1):
                        
                        found = False
                        for n in path:
                            if ((n[0] == c[0]) and (n[1] <= c[1])):
                                found = True
                                break

                        if not found:
                            valid_candidates.append(c)
                
                if (len(valid_candidates) > 0):
                    random.shuffle(valid_candidates)
                    path.extend(valid_candidates)
                
                if (index < len(path) - 1):
                    return self.pathFinding(index + 1, path)
                else:
                    return None

        
    def getCandidates(self, node):
        pos = node[0]
        count = node[1]
        
        cnd = []    
                    
        if (pos[0] - 1 >= 0):
            cnd.append(((pos[0] - 1, pos[1]), count + 1))
            
        if (pos[0] + 1 <= self.ymax):
            cnd.append(((pos[0] + 1, pos[1]), count + 1))
            
        if (pos[1] - 1 >= 0):
            cnd.append(((pos[0], pos[1] - 1), count + 1))
            
        if (pos[1] + 1 <= self.xmax):
            cnd.append(((pos[0], pos[1] + 1), count + 1))
            
        return cnd
    
    def isValid(self, matrix):
        
        is_valid = False
        
        if (matrix != None):
        
            lin = len(matrix)
            col = len(matrix[0])
            
            is_valid = (lin > 0) and (col > 0)
        
        return is_valid
            

## Teste da Solução

In [212]:
%%writefile missao3.py
from nose.tools import assert_equal


class TestGridPath(object):

    def test_grid_path(self):
        grid = Grid()
        assert_equal(grid.find_path(None), None)
        assert_equal(grid.find_path([[]]), None)
        max_rows = 8
        max_cols = 4
        matrix = [[1] * max_cols for _ in range(max_rows)]
        matrix[1][1] = 0
        matrix[2][2] = 0
        matrix[3][0] = 0
        matrix[4][2] = 0
        matrix[5][3] = 0
        matrix[6][1] = 0
        matrix[6][3] = 0
        matrix[7][1] = 0
        result = grid.find_path(matrix)
        expected = [(0, 0), (1, 0), (2, 0),
                    (2, 1), (3, 1), (4, 1),
                    (5, 1), (5, 2), (6, 2), 
                    (7, 2), (7, 3)]
        #print(matrix)
        assert_equal(result, expected)
        matrix[7][2] = 0
        result = grid.find_path(matrix)
        assert_equal(result, None)
        print('Sua solução foi executada com sucesso! Parabéns!')


def main():
    test = TestGridPath()
    test.test_grid_path()


if __name__ == '__main__':
    main()

Overwriting missao3.py


In [274]:
%run -i missao3.py

Sua solução foi executada com sucesso! Parabéns!


In [156]:
abs(-1)

1

## Fim

### Obrigado - Data Science Academy - <a href="http://facebook.com/dsacademybr">facebook.com/dsacademybr</a>